# Directed Acyclic Graph (DAG)

A DAG is a graph with the following properties:

- It is **directed**: every edge has a direction.
- It is **acyclic**: there are no cycles; you cannot start at a node and follow edges to return to it.
- Nodes may have **multiple parents**.
- A **topological ordering** always exists.

## Formal definition

A DAG is a pair $G = (V, E)$ where:

- $V$ is a set of vertices.
- $E \subseteq V \times V$ is a set of directed edges.
- There is no sequence $v_1 \to v_2 \to \cdots \to v_k \to v_1$.

## Example (Python package dependencies)
```text
    A
   / \
  B   C
   \ /
    D
```

```text
fastapi
 ├── pydantic
 │    └── typing-extensions
 └── starlette
      └── typing-extensions
```


## Why DAGs matter

- They model **dependencies**.
- They support **topological sorting**.
- They prevent **cyclic dependencies**.

In [1]:
graph = {
    "fastapi": ["pydantic", "starlette"],
    "pydantic": ["typing-extensions"],
    "starlette": ["typing-extensions"],
    "typing-extensions": []
}

In [2]:
def dfs(node, graph, visited):
    if node in visited:
        return
    visited.add(node)

    # process node here
    print(node)

    for child in graph[node]:
        dfs(child, graph, visited)

visited = set()
dfs("fastapi", graph, visited)

fastapi
pydantic
typing-extensions
starlette


In [ ]:
from collections import deque, defaultdict

def topo_sort(graph):
    indeg = defaultdict(int)
    for u, nbrs in graph.items():
        indeg.setdefault(u, 0)
        for v in nbrs:
            indeg[v] += 1
    q = deque([n for n, d in indeg.items() if d == 0])
    order = []
    while q:
        u = q.popleft()
        order.append(u)
        for v in graph.get(u, []):
            indeg[v] -= 1
            if indeg[v] == 0:
                q.append(v)
    if len(order) != len(indeg):
        raise ValueError("cycle detected")
    return order

topo_sort(graph)

In [ ]:
def has_cycle(graph):
    WHITE, GRAY, BLACK = 0, 1, 2
    color = {node: WHITE for node in graph}

    def dfs(node):
        color[node] = GRAY
        for nbr in graph[node]:
            if color.get(nbr, WHITE) == GRAY:
                return True
            if color.get(nbr, WHITE) == WHITE and dfs(nbr):
                return True
        color[node] = BLACK
        return False

    return any(dfs(node) for node in graph if color[node] == WHITE)

has_cycle(graph)

Use cases: build systems, dependency graphs, task schedulers, version histories. Try:
- Add an extra edge to create a cycle and confirm `has_cycle` raises/returns True.
- Print the topological order and compare to `pip list --format=json` dependency graphs.